In [ ]:
import nltk
from numpy.linalg import norm
from gensim.models import FastText
import sys
import os.path as op
import random
import pandas as pd
import numpy as np 
        
        
basename = r"path"
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
from tweet_utils import tweet_vectorizer

from preprocess import Preprocess
prep = Preprocess()

        
def preprocess_tweet(tweet):
    tweet = prep.replace_contractions(tweet).lower().replace("hba1c","hbaonec").replace("a1c","aonec")
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.remove_repeating_characters(tweet)
    tweet = prep.remove_repeating_words(tweet)
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = prep.remove_non_ascii(tweet)
    tweet = prep.replace_numbers(tweet)
    tweet = prep.remove_stopwords(tweet, include_personal_words=False, include_negations=False)
    tweet = [word for word in tweet if word not in ["diabetes", "diabetic"]]
    return tweet

In [ ]:
data = pd.read_csv(r"path_to_tweet_file")
data=data.drop(["Unnamed: 0","index","level_0"],axis=1)
model = FastText.load(r"path_fasttext_modeé")
print(data.shape)

#Vectorize
data["text_vec"] = data.text.map(lambda tweet: tweet_vectorizer(preprocess_tweet(tweet), model))
data["prep"] = data.text.map(lambda tweet: preprocess_tweet(tweet))
print(data.columns)

In [ ]:
data.groupby(by="user_name").count().shape

In [ ]:
from tqdm import tqdm
vectors=np.zeros((len(data.index),300))
for i in tqdm(range(vectors.shape[0])):
    vectors[i,:] = data["text_vec"][i]
vectors

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

# Regions

In [ ]:
#Regions
EAST_ASIA_AND_PACIFIC = ["Samoa","South Korea","Philippines","Australia","Laos","Brunei","Darussalam","Macao","Singapore","Cambodia","Malaysia","Solomon Islands","China","Marshall Islands","Taiwan","Fiji","Micronesia (country)","Thailand","French Polynesia","Mongolia","Timor-Leste","Guam","Myanmar","Papua New Guinea","Hong Kong","Nauru","Tonga","Indonesia","New Caledonia","Tuvalu","Japan","New Zealand","Vanuatu","Kiribati","Northern Mariana Islands","Vietnam","North Korea","Palau"]
EUROPE_AND_CENTRAL_ASIA=["Albania","Gibraltar","Norway","Andorra","Greece","Poland","Armenia","Greenland","Portugal","Austria","Hungary","Romania","Azerbaijan","Iceland","Russia","Belarus","Ireland","San Marino","Belgium","Isle of Man","Serbia","Bosnia and Herzegovina","Italy","Slovakia","Bulgaria","Kazakhstan","Slovenia","Channel Islands","Kosovo","Spain","Croatia","Kyrgyzstan","Sweden","Cyprus","Latvia","Switzerland","Czechia","Liechtenstein","Tajikistan","Denmark","Lithuania","Turkey","Estonia","Luxembourg","Turkmenistan","Faroe Islands","Moldova","Ukraine","Finland","Monaco","United Kingdom","France","Montenegro","Uzbekistan","Georgia","Netherlands","Germany","North Macedonia"]
LATIN_AMERICA_AND_THE_CARIBBEAN=["Antigua and Barbuda","Curacao","Paraguay","Argentina","Dominica","Peru","Aruba","Dominican Republic","Puerto Rico","Bahamas","Ecuador","Sint Maarten","Barbados","El Salvador","Saint Kitts and Nevis","Belize","Grenada","Saint Lucia","Bolivia","Guatemala","Saint Martin","Brazil","Guyana","Saint Vincent and the Grenadines","British Virgin Islands","Haiti","Suriname","Cayman Islands","Honduras","Trinidad and Tobago","Chile","Jamaica","Turks and Caicos Islands","Colombia","Mexico","Uruguay","Costa Rica","Nicaragua","Venezuela","Cuba","Panama","Virgin Islands"]
MIDDLE_EAST_AND_NORTH_AFRICA = ["Algeria","Jordan","Qatar","Bahrain","Kuwait","Saudi Arabia","Djibouti","Lebanon","Syria","Syrian Arab Republic","Egypt","Libya","Tunisia","Iran","Malta","United Arab Emirates","Iraq","Morocco","West Bank and Gaza","Israel","Oman","Yemen","Palestine"]
NORTH_AMERICA = ["Bermuda","Canada","United States"]
SOUTH_ASIA = ["Afghanistan","India","Pakistan","Bangladesh","Maldives","Sri Lanka","Bhutan","Nepal"]
SUB_SAHARAN_AFRICA=["Angola","Ethiopia","Niger","Benin","Gabon","Nigeria","Botswana","Gambia","Rwanda","Burkina Faso","Ghana","São Tomé and Principe","Burundi","Guinea","Senegal","Cameroon","Kenya","Sierra Leone","Central African Republic","Lesotho","Somalia","Chad","Liberia","South Africa","Comoros","Madagascar","South Sudan","Democratic Republic of the Congo","Malawi","Sudan","Mali","Tanzania","Côte d'Ivoire","Mauritania","Togo","Equatorial Guinea","Mauritius","Uganda","Eritrea","Mozambique","Zambia","Eswatini","Namibia","Zimbabwe"]

In [ ]:
#Associate country with region
data["region"]=""
a=[]
for i in tqdm(range(len(data.index))):
    if data["country"][i] in EAST_ASIA_AND_PACIFIC:
        data["region"][i]="EAST_ASIA_AND_PACIFIC"
    elif data["country"][i] in EUROPE_AND_CENTRAL_ASIA:
        data["region"][i]="EUROPE_AND_CENTRAL_ASIA"
    elif data["country"][i] in LATIN_AMERICA_AND_THE_CARIBBEAN:
        data["region"][i]="LATIN_AMERICA_AND_THE_CARIBBEAN"
    elif data["country"][i] in MIDDLE_EAST_AND_NORTH_AFRICA:
        data["region"][i]="MIDDLE_EAST_AND_NORTH_AFRICA"
    elif data["country"][i] in NORTH_AMERICA:
        data["region"][i]="NORTH_AMERICA"
    elif data["country"][i] in SOUTH_ASIA:
        data["region"][i]="SOUTH_ASIA"
    elif data["country"][i] in SUB_SAHARAN_AFRICA:
        data["region"][i]="SUB_SAHARAN_AFRICA"
    else:
        a.append(i) #List to check if a country is missing

In [ ]:
data_EAST_ASIA_AND_PACIFIC = data[data["region"]=="EAST_ASIA_AND_PACIFIC"]
data_EUROPE_AND_CENTRAL_ASIA = data[data["region"]=="EUROPE_AND_CENTRAL_ASIA"]
data_LATIN_AMERICA_AND_THE_CARIBBEAN =  data[data["region"]=="LATIN_AMERICA_AND_THE_CARIBBEAN"]
data_MIDDLE_EAST_AND_NORTH_AFRICA =  data[data["region"]=="MIDDLE_EAST_AND_NORTH_AFRICA"]
data_NORTH_AMERICA =  data[data["region"]=="NORTH_AMERICA"]
data_SOUTH_ASIA =  data[data["region"]=="SOUTH_ASIA"]
data_SUB_SAHARAN_AFRICA =  data[data["region"]=="SUB_SAHARAN_AFRICA"]

In [ ]:
vectors_EAST_ASIA_AND_PACIFIC = vectors[data_EAST_ASIA_AND_PACIFIC.index.tolist(),:]
vectors_EUROPE_AND_CENTRAL_ASIA = vectors[data_EUROPE_AND_CENTRAL_ASIA.index.tolist(),:]
vectors_LATIN_AMERICA_AND_THE_CARIBBEAN = vectors[data_LATIN_AMERICA_AND_THE_CARIBBEAN.index.tolist(),:]
vectors_MIDDLE_EAST_AND_NORTH_AFRICA = vectors[data_MIDDLE_EAST_AND_NORTH_AFRICA.index.tolist(),:]
vectors_NORTH_AMERICA = vectors[data_NORTH_AMERICA.index.tolist(),:]
vectors_SOUTH_ASIA = vectors[data_SOUTH_ASIA.index.tolist(),:]
vectors_SUB_SAHARAN_AFRICA = vectors[data_SUB_SAHARAN_AFRICA.index.tolist(),:]

## EAST_ASIA_AND_PACIFIC

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

data_EAST_ASIA_AND_PACIFIC["label"]=""
list_silhouette=[]
list_index=[]
for i in range(4,25):
    labels=fast_kmeans.kmeans(vectors_EAST_ASIA_AND_PACIFIC, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_EAST_ASIA_AND_PACIFIC["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_EAST_ASIA_AND_PACIFIC["text_vec"].values.tolist()), data_EAST_ASIA_AND_PACIFIC["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_EAST_ASIA_AND_PACIFIC, 8, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_EAST_ASIA_AND_PACIFIC["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_EAST_ASIA_AND_PACIFIC["text_vec"].values.tolist()), data_EAST_ASIA_AND_PACIFIC["label"].values, metric="cosine",sample_size=50000)

In [ ]:
def calc_distance_to_groupCenter(group, topN, vectorColumn="text_vec"):
    matrix_form = np.asarray(group[vectorColumn].values.tolist())
    groupCenter = np.sum(matrix_form,axis=0) / matrix_form.shape[0]
    
    group["cos_similarity_to_center"] = group["text_vec"].map(lambda vec: cosinus_similarity(vec, groupCenter))
    
    group.sort_values(by="cos_similarity_to_center", ascending=False, inplace=True)
    
    print("------")
    print("Label:", group["label"].values[0], "(Number tweets: {})".format(group.shape[0]))
    print("------")
    for i, row in group.head(topN).iterrows():
        print(row["text"])
        print()   
    
    return(group)



def print_closest_tweets_to_groupCenter(data, topN, vectorColumn="text_vec"):

    print("================")
    print("Closest tweets:")
    print("================")
    data = data.groupby(by="label", as_index=False) \
              .apply(lambda data: calc_distance_to_groupCenter(data, topN,"text_vec")) \
              .reset_index(drop=True)
    return(data)

#10 closest tweets to group center by cluster    
res1 = print_closest_tweets_to_groupCenter(data_EAST_ASIA_AND_PACIFIC, 10, "text_vec")
res1.head()

In [ ]:
data_EAST_ASIA_AND_PACIFIC=data_EAST_ASIA_AND_PACIFIC.reset_index()
data_EAST_ASIA_AND_PACIFIC["text_prep"]=""
for i in tqdm(range(len(data_EAST_ASIA_AND_PACIFIC.index))):
    data_EAST_ASIA_AND_PACIFIC["text_prep"][i]=' '.join(data_EAST_ASIA_AND_PACIFIC["prep"][i]) 

In [ ]:
#Topwords
from collections import Counter
for i in range(0,8):
    data_lab=data_EAST_ASIA_AND_PACIFIC[data_EAST_ASIA_AND_PACIFIC["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## EUROPE_AND_CENTRAL_ASIA

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

print(data_EUROPE_AND_CENTRAL_ASIA.shape)
data_EUROPE_AND_CENTRAL_ASIA["label"]=""
list_silhouette=[]
list_index=[]
for i in range(4,25):
    labels=fast_kmeans.kmeans(vectors_EUROPE_AND_CENTRAL_ASIA, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_EUROPE_AND_CENTRAL_ASIA["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_EUROPE_AND_CENTRAL_ASIA["text_vec"].values.tolist()), data_EUROPE_AND_CENTRAL_ASIA["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_EUROPE_AND_CENTRAL_ASIA, 10, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_EUROPE_AND_CENTRAL_ASIA["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_EUROPE_AND_CENTRAL_ASIA["text_vec"].values.tolist()), data_EUROPE_AND_CENTRAL_ASIA["label"].values, metric="cosine",sample_size=50000)

In [ ]:
#10 closest tweets to group center by cluster
res1 = print_closest_tweets_to_groupCenter(data_EUROPE_AND_CENTRAL_ASIA, 10, "text_vec")
res1.head()

In [ ]:
data_EUROPE_AND_CENTRAL_ASIA=data_EUROPE_AND_CENTRAL_ASIA.reset_index()
data_EUROPE_AND_CENTRAL_ASIA["text_prep"]=""
for i in tqdm(range(len(data_EUROPE_AND_CENTRAL_ASIA.index))):
    data_EUROPE_AND_CENTRAL_ASIA["text_prep"][i]=' '.join(data_EUROPE_AND_CENTRAL_ASIA["prep"][i]) 

In [ ]:
#Topwords
from collections import Counter
for i in range(0,10):
    data_lab=data_EUROPE_AND_CENTRAL_ASIA[data_EUROPE_AND_CENTRAL_ASIA["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## LATIN_AMERICA_AND_THE_CARIBBEAN

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

print(data_LATIN_AMERICA_AND_THE_CARIBBEAN.shape)
data_LATIN_AMERICA_AND_THE_CARIBBEAN["label"]=""
list_silhouette=[]
list_index=[]
for i in range(4,25):
    labels=fast_kmeans.kmeans(vectors_LATIN_AMERICA_AND_THE_CARIBBEAN, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_LATIN_AMERICA_AND_THE_CARIBBEAN["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_LATIN_AMERICA_AND_THE_CARIBBEAN["text_vec"].values.tolist()), data_LATIN_AMERICA_AND_THE_CARIBBEAN["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_LATIN_AMERICA_AND_THE_CARIBBEAN, 7, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_LATIN_AMERICA_AND_THE_CARIBBEAN["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_LATIN_AMERICA_AND_THE_CARIBBEAN["text_vec"].values.tolist()), data_LATIN_AMERICA_AND_THE_CARIBBEAN["label"].values, metric="cosine",sample_size=50000)

In [ ]:
#10 closest tweets to group center by cluster
res1 = print_closest_tweets_to_groupCenter(data_LATIN_AMERICA_AND_THE_CARIBBEAN, 10, "text_vec")
res1.head()

In [ ]:
data_LATIN_AMERICA_AND_THE_CARIBBEAN["text_prep"]=""
for i in tqdm(range(len(data_LATIN_AMERICA_AND_THE_CARIBBEAN.index))):
    data_LATIN_AMERICA_AND_THE_CARIBBEAN["text_prep"][i]=' '.join(data_LATIN_AMERICA_AND_THE_CARIBBEAN["prep"][i]) 

In [ ]:
#Topwords
from collections import Counter
for i in range(0,7):
    data_lab=data_LATIN_AMERICA_AND_THE_CARIBBEAN[data_LATIN_AMERICA_AND_THE_CARIBBEAN["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## MIDDLE_EAST_AND_NORTH_AFRICA

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

print(data_MIDDLE_EAST_AND_NORTH_AFRICA.shape)
data_MIDDLE_EAST_AND_NORTH_AFRICA["label"]=""
list_silhouette=[]
list_index=[]
for i in range(4,25):
    labels=fast_kmeans.kmeans(vectors_MIDDLE_EAST_AND_NORTH_AFRICA, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_MIDDLE_EAST_AND_NORTH_AFRICA["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_MIDDLE_EAST_AND_NORTH_AFRICA["text_vec"].values.tolist()), data_MIDDLE_EAST_AND_NORTH_AFRICA["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_MIDDLE_EAST_AND_NORTH_AFRICA, 4, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_MIDDLE_EAST_AND_NORTH_AFRICA["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_MIDDLE_EAST_AND_NORTH_AFRICA["text_vec"].values.tolist()), data_MIDDLE_EAST_AND_NORTH_AFRICA["label"].values, metric="cosine",sample_size=50000)

In [ ]:
#10 closest tweets to group center by cluster
res1 = print_closest_tweets_to_groupCenter(data_MIDDLE_EAST_AND_NORTH_AFRICA, 10, "text_vec")
res1.head()

In [ ]:
data_MIDDLE_EAST_AND_NORTH_AFRICA=data_MIDDLE_EAST_AND_NORTH_AFRICA.reset_index()
data_MIDDLE_EAST_AND_NORTH_AFRICA["text_prep"]=""
for i in tqdm(range(len(data_MIDDLE_EAST_AND_NORTH_AFRICA.index))):
    data_MIDDLE_EAST_AND_NORTH_AFRICA["text_prep"][i]=' '.join(data_MIDDLE_EAST_AND_NORTH_AFRICA["prep"][i]) 

In [ ]:
#Topwords
from collections import Counter
for i in range(0,7):
    data_lab=data_MIDDLE_EAST_AND_NORTH_AFRICA[data_MIDDLE_EAST_AND_NORTH_AFRICA["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## SOUTH_ASIA

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

print(data_SOUTH_ASIA.shape)
data_SOUTH_ASIA["label"]=""
list_silhouette=[]
list_index=[]
for i in range(4,25):
    labels=fast_kmeans.kmeans(vectors_SOUTH_ASIA, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_SOUTH_ASIA["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_SOUTH_ASIA["text_vec"].values.tolist()), data_SOUTH_ASIA["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_SOUTH_ASIA, 6, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_SOUTH_ASIA["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_SOUTH_ASIA["text_vec"].values.tolist()), data_SOUTH_ASIA["label"].values, metric="cosine",sample_size=50000)

In [ ]:
#10 closest tweets to group center by cluster
res1 = print_closest_tweets_to_groupCenter(data_SOUTH_ASIA, 10, "text_vec")
res1.head()

In [ ]:
data_SOUTH_ASIA=data_SOUTH_ASIA.reset_index()
data_SOUTH_ASIA["text_prep"]=""
for i in tqdm(range(len(data_SOUTH_ASIA.index))):
    data_SOUTH_ASIA["text_prep"][i]=' '.join(data_SOUTH_ASIA["prep"][i]) 

In [ ]:
#Topwords
from collections import Counter
for i in range(0,7):
    data_lab=data_SOUTH_ASIA[data_SOUTH_ASIA["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## SUB_SAHARAN_AFRICA

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

print(data_SUB_SAHARAN_AFRICA.shape)
data_SUB_SAHARAN_AFRICA["label"]=""
list_silhouette=[]
list_index=[]
for i in range(4,25):
    labels=fast_kmeans.kmeans(vectors_SUB_SAHARAN_AFRICA, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_SUB_SAHARAN_AFRICA["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_SUB_SAHARAN_AFRICA["text_vec"].values.tolist()), data_SUB_SAHARAN_AFRICA["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_SUB_SAHARAN_AFRICA, 4, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_SUB_SAHARAN_AFRICA["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_SUB_SAHARAN_AFRICA["text_vec"].values.tolist()), data_SUB_SAHARAN_AFRICA["label"].values, metric="cosine",sample_size=50000)

In [ ]:
#10 closest tweets to group center by cluster
res1 = print_closest_tweets_to_groupCenter(data_SUB_SAHARAN_AFRICA, 10, "text_vec")
res1.head()

In [ ]:
data_SUB_SAHARAN_AFRICA["text_prep"]=""
for i in tqdm(range(len(data_SUB_SAHARAN_AFRICA.index))):
    data_SUB_SAHARAN_AFRICA["text_prep"][i]=' '.join(data_SUB_SAHARAN_AFRICA["prep"][i]) 

In [ ]:
#Topwords
from collections import Counter
for i in range(0,9):
    data_lab=data_SUB_SAHARAN_AFRICA[data_SUB_SAHARAN_AFRICA["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## NORTH_AMERICA

In [ ]:
from sklearn.metrics import silhouette_score
import fast_kmeans
from matplotlib import pyplot as plt
MAX_ITERATIONS = 150

print(data_NORTH_AMERICA.shape)
data_NORTH_AMERICA["label"]=""
list_silhouette=[]
list_index=[]
for i in range(5,25):
    labels=fast_kmeans.kmeans(vectors_NORTH_AMERICA, i, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
    data_NORTH_AMERICA["label"] = labels
    silhouette_avg=silhouette_score(np.asarray(data_NORTH_AMERICA["text_vec"].values.tolist()), data_NORTH_AMERICA["label"].values, metric="cosine",sample_size=50000)
    list_silhouette.append(silhouette_avg)
    list_index.append(i)
plt.plot(list_index,list_silhouette)

In [ ]:
labels=fast_kmeans.kmeans(vectors_NORTH_AMERICA, 18, MAX_ITERATIONS, distance="cosinus", vectorColumn="text_vec")
data_NORTH_AMERICA["label"] = labels
silhouette_avg=silhouette_score(np.asarray(data_NORTH_AMERICA["text_vec"].values.tolist()), data_NORTH_AMERICA["label"].values, metric="cosine",sample_size=50000)

In [ ]:
#10 closest tweets to group center by cluster
res1 = print_closest_tweets_to_groupCenter(data_NORTH_AMERICA, 10, "text_vec")
res1.head()

In [ ]:
data_NORTH_AMERICA=data_NORTH_AMERICA.reset_index()
data_NORTH_AMERICA["text_prep"]=""
for i in tqdm(range(len(data_NORTH_AMERICA.index))):
    data_NORTH_AMERICA["text_prep"][i]=' '.join(data_NORTH_AMERICA["prep"][i])

In [ ]:
#Topwords
from collections import Counter
for i in range(0,18):
    data_lab=data_NORTH_AMERICA[data_NORTH_AMERICA["label"]==i]
    print("Label ",i,":",Counter(" ".join(data_lab["text_prep"]).split()).most_common(15))

## Generate data

In [ ]:
tweets=pd.concat([data_EAST_ASIA_AND_PACIFIC,data_EUROPE_AND_CENTRAL_ASIA,data_LATIN_AMERICA_AND_THE_CARIBBEAN,data_MIDDLE_EAST_AND_NORTH_AFRICA,data_NORTH_AMERICA,data_SOUTH_ASIA,data_SUB_SAHARAN_AFRICA])

In [ ]:
tweets=tweets.drop(["Unnamed: 0.1","Unnamed: 0.1.1"],axis=1)

In [ ]:
tweets.to_csv(r"path")